### tuto gpd bokeh

https://automating-gis-processes.github.io/CSC18/lessons/L5/advanced-bokeh.html

In [3]:
import geopandas as gpd
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource
from pprint import pprint

In [2]:
path = 'map/departements-20180101.shp'

In [3]:
gdf = gpd.read_file(path)

In [4]:
r = 'tuto-bokeh/dataE5/dataE5/'
a = gpd.read_file(r + 'addresses.shp')
b = gpd.read_file(r + 'roads.shp')

# Reproject to the same projection
CRS = b.crs
a = a.t# Reproject to the same projection
CRS = roads.crs
addresses = addresses.to_crs(crs=CRS)
o_crs(crs=CRS)

AttributeError: 'GeoDataFrame' object has no attribute 't'

In [ ]:
# Convert GeoDataFrames into GeoJSONDataSource objects (similar to ColumnDataSource)
map_source = GeoJSONDataSource(geojson=gdf.to_json())

p = figure(title='A test Map')

In [ ]:
point_source = GeoJSONDataSource(geojson=a.to_json())
roads_source = GeoJSONDataSource(geojson=b.to_json())

In [ ]:
p.multi_line('xs', 'ys', source=roads_source, color='gray', line_width=3)

In [ ]:
p.circle('x', 'y', source=point_source, color='black', size=6)

In [ ]:
show(p)

### filtering only bourg fc departement

In [34]:
dep_path = 'map/departements-20180101.shp'
departement = gpd.read_file(dep_path)
#filtering only on some insee code
codes = [21, 25, 39, 58, 70, 71, 89, 90]
codes = [str(x) for x in codes]

In [37]:
departement = departement.loc[departement.code_insee.isin(codes)]

In [38]:
departement.to_file('data4/bourgogne_fc.shp')

### end tuto and start with the real map

In [1]:
import geopandas as gpd
from bokeh.plotting import figure, show
from bokeh.models import GeoJSONDataSource, HoverTool, Div, Select, RadioGroup, Paragraph, MultiChoice
from pprint import pprint
from shapely.geometry import Point
import pandas as pd
from bokeh.layouts import row, column

In [2]:
departement = gpd.read_file('data4/bourgogne_fc.shp')

In [3]:
departement.head(n=2)

,code_insee,nom,nuts3,wikipedia,surf_km2,geometry
0,71,Saône-et-Loire,FR263,fr:Saône-et-Loire,8613.0,"POLYGON ((3.62229 46.74091, 3.62230 46.74103, ..."
1,89,Yonne,FR264,fr:Yonne (département),7461.0,"POLYGON ((2.84849 47.72603, 2.84963 47.72627, ..."


In [4]:
departement_source = GeoJSONDataSource(geojson=departement.to_json())

In [5]:
CRS = departement.crs

In [6]:
ght = pd.read_csv('data4/ght.csv', index_col=0)

In [7]:
coord = pd.read_csv('data4/coord_with_cities.csv', index_col=0)

In [8]:
coordght= pd.merge(coord, ght, how='left')

In [9]:
geometry = [Point(xy) for xy in zip(coordght.longitude, coordght.latitude)]
etablissement = gpd.GeoDataFrame(coordght.drop(['longitude', 'latitude'], axis=1), crs=CRS, geometry=geometry)

In [10]:
etablissement_source = GeoJSONDataSource(geojson=etablissement.to_json()) 

In [11]:
p = figure(
    title='New test map',
    x_axis_location=None, 
    y_axis_location=None,
    #sizing_mode='stretch_width',
    width=1200,
    height=750,
    #tooltips=TOOLTIPS
)

In [12]:
patches = p.patches('xs', 
          'ys', 
          source=departement_source,
          fill_color='blue',
          fill_alpha=0.5, 
          line_color="black", 
          line_width=0.5)

In [13]:
circles = p.circle(
    'x',
    'y',
    source=etablissement_source,
    color='red', 
    size=6,
    hover_color='yellow'
)

In [14]:
TOOLTIPS = [
    ('(x, y)', '($x, $y)'),
    ('etablissement', '@etablissement'),
    ('village', '@village'),
    ('ght', '@nom_ght')
]

In [15]:
ht = HoverTool(
    tooltips= TOOLTIPS,
    renderers = [circles]
)

In [16]:
p.tools.append(ht)

In [17]:
#adding widgets
div = Div(
    text="""
        <p>Select the GHT</p>
        """,
    width=200,
    height=30,
)

In [18]:
#Filtrer par entrant / sortant
LABELS = ['Entrant', 'Sortant', 'Entrant & sortant']
radio_group = RadioGroup(labels=LABELS, active=0)
'''
radio_group.js_on_click(CustomJS(code="""
    console.log('radio_group: active=' + this.active, this.toString())
"""))
'''

'\nradio_group.js_on_click(CustomJS(code="""\n    console.log(\'radio_group: active=\' + this.active, this.toString())\n"""))\n'

In [19]:
#filter by ght

ght_names = ght.nom_ght.drop_duplicates().sort_values().tolist()
select = Select(title="GHT:", value=ght_names[0], options=ght_names)
'''
select.js_on_change("value", CustomJS(code="""
    console.log('select: value=' + this.value, this.toString())
"""))
'''

'\nselect.js_on_change("value", CustomJS(code="""\n    console.log(\'select: value=\' + this.value, this.toString())\n"""))\n'

In [57]:
etablissement_source

GeoJSONDataSource(id='1003', ...)

In [46]:
pprint(etablissement_source.geojson)

('{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", '
 '"properties": {"adresse": "18 Bis Rue Pierre Semard", "code_postal": 89400, '
 '"departement": "Bourgogne-Franche-Comt\\u00e9", "etablissement": "AIHP '
 'CENTRE ARMANCON", "index": 0, "municipalite": "Auxerre", "nom_ght": null, '
 '"village": "Migennes", "ville": null}, "geometry": {"type": "Point", '
 '"coordinates": [3.53041, 47.95981]}}, {"id": "1", "type": "Feature", '
 '"properties": {"adresse": "45 Bd Henri Bazin", "code_postal": 21000, '
 '"departement": "Bourgogne-Franche-Comt\\u00e9", "etablissement": "C.R.B LES '
 'ROSIERS", "index": 1, "municipalite": "Dijon", "nom_ght": null, "village": '
 'null, "ville": "Dijon"}, "geometry": {"type": "Point", "coordinates": '
 '[5.019, 47.30041]}}, {"id": "2", "type": "Feature", "properties": '
 '{"adresse": "4 Rue \\u00c9tienne Litaud", "code_postal": 58000, '
 '"departement": "Bourgogne-Franche-Comt\\u00e9", "etablissement": "CENTRE '
 'COLBERT MEDECINE ET 

In [42]:
help(etablissement_source)

Help on GeoJSONDataSource in module bokeh.models.sources object:

class GeoJSONDataSource(ColumnarDataSource)
 |  GeoJSONDataSource(*args, **kwargs) -> 'Model'
 |  
 |  Method resolution order:
 |      GeoJSONDataSource
 |      ColumnarDataSource
 |      DataSource
 |      bokeh.model.model.Model
 |      bokeh.core.has_props.HasProps
 |      bokeh.model.util.HasDocumentRef
 |      bokeh.util.callback_manager.PropertyCallbackManager
 |      bokeh.util.callback_manager.EventCallbackManager
 |      builtins.object
 |  
 |  Data descriptors defined here:
 |  
 |  geojson
 |      GeoJSON that contains features for plotting. Currently
 |      ``GeoJSONDataSource`` can only process a ``FeatureCollection`` or
 |      ``GeometryCollection``.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  __overridden_defaults__ = {}
 |  
 |  __properties__ = {'geojson': <bokeh.core.property.nullable.NonNullable...
 |  
 |  __q

In [ ]:
callback = CustomJS(args=dict(source=etablissement_source), code="""
        const data = source.data;
        const f = cb_obj.value
        
        for (let i = 0; i < x.length; i++) {
            y[i] = Math.pow(x[i], f)
        }
        source.change.emit();
    """)
select.on_change('value', callback)

In [20]:
para = Paragraph(
    text="""Your text is initialized with the 'text' argument.  The
    remaining Paragraph arguments are 'width' and 'height'. For this example, those values
    are 200 and 100, respectively.""",
    width=200, height=100
)

In [21]:
etab_names = coord.etablissement.drop_duplicates().tolist()

mc = MultiChoice(value=etab_names[0], options=etab_names)
'''
multi_choice.js_on_change("value", CustomJS(code="""
    console.log('multi_choice: value=' + this.value, this.toString())
"""))
'''

'\nmulti_choice.js_on_change("value", CustomJS(code="""\n    console.log(\'multi_choice: value=\' + this.value, this.toString())\n"""))\n'

In [24]:
widgets = column(div, select, radio_group, multi_choice, para)
layout = row(p, widgets)

In [23]:
show(layout)